<a href="https://colab.research.google.com/github/Hectooooooor/AI/blob/main/3.0704_Colab_LINE_Bot_with_GEMINI_Tooluse_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [7]:
import requests
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token.strip()}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://liz-magnetostrictive-discomposingly.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://liz-magnetostrictive-discomposingly.ngrok-free.dev


True

In [8]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [9]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [10]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。學校秉持「在明明德，在新民，在止於至善」的精義，以培育具備專業技能、團隊精神、人文素養與宏觀服務的科技人才為目標。

**歷史沿革：**
明新科技大學的創立可追溯至1966年3月1日，當時以「明新工業專科學校」立案，設有機械、土木、工業管理等三科，首屆招收300名學生。隨著發展，學校陸續增設多個科系。1997年7月，奉教育部核准改制為「明新技術學院」並附設專科部。最終在2002年8月，獲教育部核准升格為「明新科技大學」。2018年12月，學校更名為「明新學校財團法人明新科技大學」。

**學術與特色：**
明新科大現設有半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院等六個學院，涵蓋20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。學校的辦學定位為「一流產業大學」，致力於培養「跨域整合、務實創新與全人學習」的專業人才。

特別值得一提的是，明新科大積極配合產業趨勢，於2021年整合相關學系成立了「半導體學院」。2022年，學校獲得教育部核准通過「半導體科技博士學位學程」，這是其創校以來第一個博士班，標誌著學校邁向新的里程碑。明新科大在半導體產業人才培育方面表現突出，曾於2022年《遠見》雜誌「企業最愛公私立技職科大調查」中，在半導體相關領域獲得企業高度青睞，起薪排名私校第一，甚至優於部分國立科大。根據1111人力銀行統計，明新科大在半導體產業界最愛聘用的畢業生中名列前茅，與頂尖大學並列前五大，是唯一入榜的私立科大。

學校位於新竹縣新豐鄉，坐擁新竹科學園區與新竹工業區的豐富產業資源，地利之便使其能強化產學鏈結，提供學生多元的實習與就業機會。明新科大也積極推動「永續智慧商務」的教學與實習場域，導入生成式AI與AI專案應用，發展智慧零售、智慧金融、智慧製造與智慧商業等實驗室，以培養學生具備與業界同步接軌的能力。


In [11]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學的現任校長是**呂明峯**教授。他於2025年2月1日正式上任，為明新科技大學的第11任校長。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U7d9a3890d4673e922827d2dda79ff019","events":[{"type":"message","message":{"type":"text","id":"595532594931040785","quoteToken":"trM-6PESLFPDExx5spz9XE_7OG8u-ev56YW1FcJAG3oTPuCw4mqZzaFg87_-GPx5zZXcIelmMhU8Niu1Od9KLjFWDxtmYsnGXei8fbdQjp0PwG0vrDUy7nKtx6CytseuoWke99KhDMGi51zTPGbR-Q","markAsReadToken":"zt8U5HCqnw82Ls9ydBW6fZIj0WTUQagqFsxRXyoiduhSQEI4zTWGOCk45HBRyHuUCQxcGk7KhmQrxYzkEAU5P9Suud_15CCjci6TOSzpAYr2KhTF57RuEdY8VAFG-kvym9RurZTYI2UtbkAN_RVEmWKuIcxQ_AkcH2g2RWGUgbSzpcETNOL_PbIUyifE_Es93FhiP3FpTxfOdaeDATjEyQ","text":"AI 校長是誰"},"webhookEventId":"01KECE0G7MF9XET0NBS46E3CFZ","deliveryContext":{"isRedelivery":false},"timestamp":1767796391741,"source":{"type":"user","userId":"Ue020f480d3b30476c7126cc8cf6811c0"},"replyToken":"7707a997cd1e499abec93f345a3b1f51","mode":"active"}]}


ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1799486110.py", line 39, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.12/di